# `ad39_package` Documentation

Team39: Philipp Arens, Ben Ray, Andy Zhuo

## Introduction

Derivatives play an essential role in many areas of science and engineering such as fluid dynamics, solving ODEs, optimization and - relatedly - machine learning. In general, there exist four main ways of computing derivatives in practice, that is: manual computation (i.e. computing and implementing derivatives by hand), symbolic differentiation (i.e. WolframAlpha or SymPy), numerical differentiation (i.e. finite difference method) and automatic differentiation (AD). Depending on the complexity of the target function, manual differentiation can require significant amounts of time and is often challenging to "debug". Symbolic differentiation alleviates this at the expense of potentially bloated/unitutitive expressions. If analytic derivative expressions are not needed, numerical differentiation, that is approximating derivatives through finite differences, could be considered. This, however, can lead to approximation/floating point errors if the step size is chosen inadequately. 

AD has emerged as a promising way of adressing many of these issues. Though not providing closed form/symbolic expressions, it allows us to compute derivatives with machine precision without introducing large computational overhead. Sparked by recent advances in machine learning, in particular deep neural networks, which use a subclass of AD (i.e. the backpropagation algorithm) in their training phase, AD has shown its potential across a variety of different aplications.

In this project we have developed a software package implementing thus far a subclass of AD, namely forward mode AD. We aim at making this software intuitive to use, following best practices in terms of relying on the python data model and style guides.

## Background

Central to automatic differentiation (AD) is use of the chain rule to decompose derivatives into elemtary binary operations (+, -, *, /, **) and unary operations (sin, cos, exp, log, etc.) Using the chain rule allows for two modes of evaluating a derivative:

1. Forward mode AD - the derivative is computed "bottom up" (i.e. fixing the independent variable(s), take the derivative of the innermost function first, and then move up taking derivatives of each sub-expression recursively)
2. Reverse mode AD - the derivative is computed "top down" (i.e. you fix the dependent variable, take the derivative of the outermost function first with respect to its sub-expressions, and then move inwards taking successive derivatives with respect to their inner sub-expressions).

AD can be generalized to both oridinary and partial derivatives using matrix products of Jacobians.

Finally, in forward mode AD, we commonly redefine our arithmetic to use dual numbers of the form $x + x'\epsilon$ where $\epsilon$ is a mathematical object not contained in the set of real numbers, with the property $\epsilon^2 = 0$. This allows us to store the decomposed derivative as a computational graph represented by dual numbers in each node. Despite this abstract implementation, we can still preserve the critical concept of "duck typing": as long as our dual numbers act like an input to their function (e.g. they can be added, multiplied, etc.) then the function will work.

## How to use `ad39_package`

### Milestone 2 Installation

The following instructions can be executed by copying the file `m2-install.sh` from Team39's GitHub repository and executing it locally by running `source m2-install.sh`. Alternatively, the following instructions can be run line by line.

In this setup procedure, we first create a virtual environment in which to install the `ad39_package` and its dependencies (optional), before explaining how to import and use the package's functionality.

A user can create a simple virtual environment and activate it as follows:

    python3 -m venv test_env
    source test_env/bin/activate

Now, uers will be able to install our package by cloning Team39's project repository onto their local machine via the command:

    git clone git@code.harvard.edu:CS107/team39.git

Before importing from the package users will also have to install its dependencies - at this stage only the `numpy` package package via:

    pip install numpy

Finally, users will change into the `src` directory and either open the Python interpreter or create scripts here than can import the `ad39_package`:

    cd src 
    python3

Users will now be able to import and use our package as per the following Python script, which also showcases its functionality:

In [ ]:
# Import the `ad39_package`
import ad39_package as ad39

# Instantiate an object for performing AutoDiff by calling AD with the desired function
ADtest = ad39.AD(lambda x: 4 * ad39.sqrt(x) ** 3)

# Evaluate the function at x = 4 by calling the `eval(x)` method with argument 4
result = ADtest.eval(4)
print(f'f(4) = {result}') # result = 32

# Compute the derivative evaluated at 4 by calling the `forward_mode(x)` method with argument 4
deriv = ADtest.forward_mode(4)
print(f'f\'(4) = {deriv}') # deriv = 12

# Compute the Jacobian evaluated at 2 by calling the `jacobian(x)` method with argument pi/4
jacob = ADtest.jacobian(4)
print(f'The Jacobian of f(4) is {jacob}.') # jacob = 12

### Future usage (beyond Milestone 2)

In future, our package will be available from the Test PyPi server using pip. Users will install the `ad39_package` and its dependencies using a command similar to the following:

    python3 -m pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/ ad39_package

Having installed the package, users will be able to call `import ad39_package as ad39`, before using the package to its full effect.

## Software Organization

### Structure

    team39
        ├── LICENSE
        ├── README.md
        ├── docs
        │   ├── milestone1.ipynb
        │   ├── milestone2.ipynb
        │   └── milestone2_progress.md
        ├── m2-install.sh
        ├── project.toml
        ├── setup.cfg
        ├── src
        │   └── ad39_package
        │       ├── __init__.py
        │       ├── core.py
        │       └── helpers.py   
        └── tests
            ├── check_coverage.py
            ├── check_coverage.sh
            ├── run_tests.sh
            ├── test_core.py
            └── test_helpers.py
            

### Modules

Most of the file tree is self explanatory, but to briefly explain the contents of the `ad39_package`:
- `__init__.py` will initialize the entire package.
- `core.py` contains the base class for performing AD, namely `AD`, with methods for evaluating a function, performing forward mode, and computing the Jacobian.
- `helpers.py` contains the `DualNumber` class along with operator overloading for elementary unary and binary operations, which facilitates forward mode AD but is not accessible by users. Additionally it contains many functions available to the user for use _within a function_, e.g. in order to evaluate `sin(pi/2)`, a user must first define a function such as `lambda x : ad39.sin(x)` before using the `eval(x)` method from the `ad39.AD` class at pi/2; `ad39.sin(pi/2)` will return a syntax error. These additional functions available to the user are `sqrt`, `sin`, `cos`, `tan`, `arcsin`, `arccos`, `arctan`, `sinh`, `cosh`, `tanh`, `log`, `exp`, `logistic`.

### Tests

Our test suite lives in the designated `tests` directory on the root level of the main `team39` directory. 

First, we test our package's functionality with pytest by executing the test driver script as follows: `./run_tests pytest`. This script in turn runs the tests we have written in `test_core.py` and `test_helpers.py` (for the `core` and `helpers` modules in `ad39_package` respectively). 

Next, we check our tests' coverage by executing the other driver script as follows: `./check_coverage pytest`. This generates a report on the coverage in `.xml` format. On Github actions we extract the coverage rate from this XML file and compare it to our 90% passing benchmark using the `check_coverage.py` script.

### Package Distribution

At this stage (Milestone 2), our package is installed by git cloning Team39's repository, as per the instructions in the "How to use `ad39_package`" above.

In future (beyond Milestone 2), we plan to distribute our package using Test PyPI (i.e PEP517/518). To this end we would use the Test PyPI server and resolve extra dependencies from the main server with the `--extra-index-url` flag.

## Implementation

### Core Data Structures

At this stage (Milestone 2), our implementation of forward mode contains little complexity in terms of data structures and mainly uses two class objects: `AD` (available to the user) and `DualNumbers` (abstracted from the user).

`AD` class instances are created for specific functions, which the user may be interested in evaluating or differentiating using forward mode. Hence, the `AD` class has just one attribute for these specific functions `self.f`. Thus, the `AD` class may be instantiated on any kind of function object, i.e. an anonymous/lambda function or non-anonymous function in Python. The `AD` class and its methods can be found in `core.py`.

The `DualNumbers` class has two key attributes `self.real` and `self.dual`. Intuitively, these describe the real and dual parts of a dual number respectively. The class also implements operator overloading for standard binary, reverse binary, and unary operators. The `DualNumber` class an its methods can be found in `helpers.py`.

### Classes and Attributes

In [ ]:
"""Full implementation in `core.py`"""

class AD:
    def __init__(self, f):
        """Initialize an instance of an automatic differentiation object given a scalar-valued function `f`."""
        self.f = f

    def eval(self, x):
        """Evaluate the value of `self.f` at `x`. Only defined for scalar-valued functions."""
        pass

    def forward_mode(self, x):
        """Compute the derivative of `self.f` evaluated at `x` using forward mode AD.
           Only defined for scalar-valued functions."""
        pass
    
    def jacobian(self, x):
        """Compute the Jacobian of `self.f` evaluated at `x` using forward mode AD.
           Only defined for scalar-valued functions."""
        pass

As the chain rule holds for dual numbers we know that for $f(z) = f(a + b \epsilon) = f(a) + f'(a)b \epsilon$. Thus, abiding to the principles of duck typing, we overload the operators for dual numbers:

In [ ]:
"""Full implementation in `helpers.py`"""

class DualNumber:
    _supported_scalars = (float, int)

    def __init__(self, real, dual=1.0):
        self.real = real
        self.dual = dual

    def __repr__(self):
        pass

    # Binary operations
    def __add__(self, other):
        pass

    def __sub__(self, other):
        pass

    def __mul__(self, other):
        pass

    def __truediv__(self, other):
        pass
    
    def __pow__(self, other):
        pass

    # Reverse binary operations
    def __radd__(self, other):
        pass

    def __rmul__(self, other):
        pass

    def __rsub__(self, other):
        pass

    def __rtruediv__(self, other):
        pass
    
    # Unary operations
    def __neg__(self):
        pass

"""More complex functions for dual numbers like `sin` / `exp` / etc. are implemented as standalone functions accessible to the user"""

### External Libraries

Only `numpy` at this stage.

## License

We decided to go with an MIT (i.e. copyright) license as it permits reuse within proprietary software, provided that all copies of the software or its substantial portions include a copy of the terms of the MIT License and also our original copyright notice. Since we do not hope to commercialize this project, we are happy for others to use it. However, we feel that including the original copyright notice in all derived projects is important, so that future developers understand that the `ad39` package was designed as an educational project (and not more than that), i.e. we want to set developers' expectations for using the `ad39` package by telling them the provenance of the package.

## Future Features (including Extension)

Our current implementation of Automatic Differentiation uses Dual Numbers to compute the derivative of a scalar function $f: \mathbb{R} \rightarrow \mathbb{R}$. While this suffices for most use cases, we will extend our package to support multivariate vector functions $f: \mathbb{R}^m \rightarrow \mathbb{R}^n$. However, as the input dimension grows increasingly large, using the forward mode variant of AD becomes inefficient as the number of passes grows proportionate to the number of inputs. Accordingly, in order to acccommodate multiple inputs while accounting for efficiency, we will also implement reverse mode for multivariate functions, which bounds the number of forward passes to one but now requires the storage of all intermediate operations as variable-nodes in a computational graph. 

The reverse mode implementation of AD relies on a directed computational graph represented as a set of instantiated `Node` objects. Each instance of the `Node` class will store, upon initialization, 
1. `children` - an array of children nodes, used to accumulate the current node's adjoint
2. `val` - the function's value evaluated at this step in the computation
3. `partials`- an array of the node's local partial derivatives with respect to the parent(s) of the node
3. `adjoint` - the function's partial derivative with respect to the current node, initialized as 0 until a reverse pass is made.

The use of a digraph data structure will allow us to traverse through each intermediate operation in a single forward pass to evaluate the function and compute the partials. Performing a single reverse pass by recursively traversing through each node and its children and using the local partials to build up the adjoints will compute the partial for each input variable and thus the function's gradient. In order to perform a forward pass on the digraph, we will need to extend the operator overloads for `DualNumber` to the `Node` class, making modifications as necessary. While the `val` and `partials` attributes will be computed similarly to `real` and `dual` in the current `DualNumber` class, we will need to extend the operator overloads for `Node` to account for the next intermediate operations, represented as the children of a node. 

Finally, if time allows, with a computational graph constructed via the implemented `Node` class, we aim to provide functionality for drawing the computational graph using the `NetworkX` Python package. 

## Appendix

### Milestone 1 Feedback (some of which no longer applies)
- Background
    - We explained the difference between forward and reverse node as computing the derivative "bottom up" and "top down" rather than "from the inside out" and "from the outside in", given the intuition that expressions can be thought of as syntax trees.
    - We clarified that $\epsilon$ is a mathematical object not contained in the set of real numbers, to clarify what we had previously described as an "abstract number".

- How To Use
    - Summarized usage in a code snippet.
    - Changed how you initialized variables as nodes, removing the need for providing a name to the node (which could conflict across variables): i.e. before you initialized as follows `x1 = Node("x1")`, now it is just `x1 = Node()`.

- Software Organization
    - Added a note that we may need more modules in `ad39_package` as the project grows.

- Implementation
    - Removed `node.child` from our implementation of the `Node` class following feedback that "as long as each node keeps reference of its parent(s), you can access the whole graph with the output nodes as a handle." Also removed `node.name` as per earlier feedback - this is no longer necessary, and if we ever need to give a unique ID to nodes, it will be done entirely on the backend.
    - Removed the unintuitive `ForwardMode` class, and instead defined an `AD` class, which now includes: (1) a new `forward_mode` method, (2) the `grad` method (moved from the `Node` class to here, but no longer a static method), and (3) any other methods that will be included in our extension, e.g. `draw` for drawing the computational graph. Note, that the `AD` class object is initialized with the function on which AD will be performed, and we may use a further `Tree` class (or just a list of `Node` objects) under the hood to store the nodes created during the computation.